In [2]:
import pandas as pd
import numpy as np
import praw
from praw.models import MoreComments
import os
from groq import Groq
import time

In [5]:
!pip install groq

In [3]:
reddit = praw.Reddit(user_agent="Extract Reddit v1.0 by u/External_Middle8912",
                     client_id="KtFyZUw2ZS0gPLVwnWpITw", 
                     client_secret="8xdnOL6gEBbwCgu389aJrj3c1hKj7w")
# url = "https://www.reddit.com/r/depression/comments/1iswzg5/i_forget_depression_isnt_common/"
category = ["depression", "anxiety", 'mentalillness']
data = []
for item in category:
    subreddit = reddit.subreddit(item)
    for submission in subreddit.hot(limit=1000):
        data.append([
            submission.title,       # Post Title
            submission.selftext,
            submission.score,       # Upvote Score
            submission.num_comments,# Number of Comments
            submission.url,         # Post URL
            submission.author,      # Author Name
            submission.created_utc,  # Timestamp (Unix)
            item
        ])
posts = pd.DataFrame(data,columns=["Title", "Text", "Score", "Comments", "URL", "Author", "Created_UTC", "Subreddit"])       
# indexNames = posts[(posts.body == '[removed]') | (posts.body == '[deleted]')].index
# posts.drop(indexNames, inplace=True)

In [4]:
posts['Text'] = posts['Text'].str.replace(r'\\r\\n|\\n|\\', " ", regex = True)

In [8]:
posts['Text'].iloc[5]

'Whether you’ve tried to end your life but didn’t quite get there or you’ve been in a near death experience, what does death feel like and what happens?\n\nI’m on the brink of suicide. I contemplate it every day. One of the few things that is holding me back is I don’t have any idea of what happens after, or how dying feels.\n\nIf anyone has tried to hang themselves, cut their wrists, or attempted suicide, or even been an accidental near death experience, and has been brought back at the last minute; can you describe the feeling, and what spiritual experience occurs when you get close to fading away?'

In [4]:
posts = pd.read_csv("/Users/bowiechuang/Documents/GitHub/baychuang/backend/data/reddit_post.csv")
posts

,Unnamed: 0,Title,Text,Score,Comments,URL,Author,Created_UTC,Subreddit
0,0,Our most-broken and least-understood rules is ...,We understand that most people who reply immed...,2366,177,https://www.reddit.com/r/depression/comments/d...,SQLwitch,1.572361e+09,depression
1,1,"Regular check-in post, with information about ...",Welcome to /r/depression's check-in post - a p...,26,247,https://www.reddit.com/r/depression/comments/1...,SQLwitch,1.727565e+09,depression
2,2,i regret ever opening up to anyone,they make you feel even shittier about what yo...,53,20,https://www.reddit.com/r/depression/comments/1...,Acrobatic_Lie7392,1.740252e+09,depression
3,3,I'm going to commit suicide in a few days,I’m 19 years old and I just can't live like th...,41,37,https://www.reddit.com/r/depression/comments/1...,Ewexz,1.740245e+09,depression
4,4,"I hate happy families, especially dad-daughter...",I feel so disappointed and sad whenever I see ...,36,6,https://www.reddit.com/r/depression/comments/1...,plushielvr,1.740246e+09,depression
...,...,...,...,...,...,...,...,...,...
2797,2797,Obsession with deceased celebrity?,"This is not a joke, please take it seriously.\...",10,6,https://www.reddit.com/r/mentalillness/comment...,tikhung01,1.736532e+09,mentalillness
2798,2798,Please help! Why am I all of a sudden angry at...,So yesterday I was haveing a good day then I R...,3,4,https://www.reddit.com/r/mentalillness/comment...,Bloody_skulls_,1.736555e+09,mentalillness
2799,2799,I had to call crisis on my manic boyfriend and...,My boyfriend has had something mentally going ...,2,30,https://www.reddit.com/r/mentalillness/comment...,Few-Recover-4973,1.736563e+09,mentalillness
2800,2800,Do you think it’s too late to tell my psychiat...,I’ve been seeing her for about a year & a half...,3,2,https://www.reddit.com/r/mentalillness/comment...,Mental-Net1389,1.736554e+09,mentalillness


## Model for Classification

In [5]:
groq_API = "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"
client = Groq(api_key=groq_API)
def categorize_reddit_content(content):
      time.sleep(3)
      messages = [
          {
              "role": "system",
              "content": 'null'
          },
          {
              "role": "user",
              "content": f"Please categorize our reddit post content into one of the three categories: 'Depression', 'Anxiety', 'Mental Illness': \n\n {content}. The output should only be a word with no extra texts."
          }
      ]

      # prompt = "Please categorize our reddit post content into one of the three categories: 'Depression', 'Anxiety', 'Mental Illness'."

      chat_completion = client.chat.completions.create(
              temperature = 0.1, 
              messages = messages, 
              model = "llama3-70b-8192", 
              n = 1, 
      )
      category = chat_completion.choices[0].message.content
      return category

# print(chat_completion.choices[0].message.content)

## Model for Giving Numbers

In [6]:
groq_API = "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"
client = Groq(api_key=groq_API)

def severity_rating(content):
      time.sleep(3)
      messages = [
          {
              "role": "system",
              "content": 'null'
          },
          {
              "role": "user",
              "content": f"On a scale from 1 to 10, rate the severity of the situation described in the content.: \n\n {content}. A rating of 1 indicates minimal or no significant signs of depression, anxiety, or mental issues, while a rating of 10 represents an extremely severe and critical situation. The output should only be a single number in between with no extra texts."
          }
      ]

      # prompt = "Please categorize our reddit post content into one of the three categories: 'Depression', 'Anxiety', 'Mental Illness'."

      chat_completion = client.chat.completions.create(
              temperature = 0.15, 
              messages = messages, 
              model = "llama3-70b-8192", 
              n = 1, 
      )
      category = chat_completion.choices[0].message.content
      return category


In [7]:
sub_df = posts[:10]
sub_df["New_Text"] = sub_df["Text"].apply(categorize_reddit_content)
sub_df["Rating"] = sub_df['Text'].apply(severity_rating)

/var/folders/hk/1l4v0gdd41z7xrwc5_b0pcmm0000gn/T/ipykernel_4665/359138803.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["New_Text"] = sub_df["Text"].apply(categorize_reddit_content)
/var/folders/hk/1l4v0gdd41z7xrwc5_b0pcmm0000gn/T/ipykernel_4665/359138803.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sub_df["Rating"] = sub_df['Text'].apply(severity_rating)


In [8]:
sub_df[['New_Text', 'Rating']]

,New_Text,Rating
0,I would categorize this Reddit post content in...,6
1,The category for this post is: Depression,1
2,Depression,8
3,Depression,10
4,Depression,7
5,Depression,9
6,Depression,10
7,Depression,10
8,Depression,8
9,Depression,8


In [9]:
sub_df1 = posts[:250]
sub_df1["New_Text"] = sub_df1["Text"].apply(categorize_reddit_content)
sub_df1["Rating"] = sub_df1['Text'].apply(severity_rating)


RateLimitError: Error code: 429 - {'error': {'message': 'Rate limit reached for model `llama3-70b-8192` in organization `org_01jmqkkgdvfeds21h90bdrzsqd` service tier `on_demand` on : Limit 500000, Used 499415, Requested 1265. Please try again in 1m57.354599999s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing', 'type': '', 'code': 'rate_limit_exceeded'}}

In [12]:
sub_df1['New_Text']

KeyError: 'New_Text'

In [ ]:
# Call this function in the notebook or when using the API
if __name__ == "__main__":
    # Test categorization with a sample post (or input from the frontend)
    category = categorize_reddit_content(input)
    print(category)  # You can format this for JSON or a suitable response
